In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings 
warnings.filterwarnings('ignore')
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler
from sklearn.cluster import KMeans

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
df = pd.read_pickle('data/latest_merged.pkl')

In [4]:
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",100)
pd.set_option("display.max_colwidth",None)

In [5]:
df.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_2,EXT_SOURCE_3,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,No_of_PreviousApplications,Credit_to_Income_Ratio,Annuity_to_Income_Ratio,GoodsPrice_to_Income_Ratio,No_of_DocumentsSubmitted,EXT_SCORES_AVG,EXT_SCORES_SUM,FLAG_CONTACT_INFO,EXT_SOURCE_1,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Other,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Security staff,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Government,ORGANIZATION_TYPE_Kindergarten,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_XNA,clusters,AMT_ANNUITY_y_list,AMT_APPLICATION_list,AMT_CREDIT_y_list,AMT_DOWN_PAYMENT_list,AMT_GOODS_PRICE_y_list,HOUR_APPR_PROCESS_START_y_list,NFLAG_LAST_APPL_IN_DAY_list,RATE_DOWN_PAYMENT_list,DAYS_DECISION_list,SELLERPLACE_AREA_list,CNT_PAYMENT_list,DAYS_FIRST_DRAWING_list,DAYS_FIRST_DUE_list,DAYS_LAST_DUE_1ST_VERSION_list,DAYS_LAST_DUE_list,DAYS_TERMINATION_list,NFLAG_INSURED_ON_APPROVAL_list,NAME_CONTRACT_TYPE_y_list_Cash loans,NAME_CONTRACT_TYPE_y_list_Consumer loans,NAME_CONTRACT_TYPE_y_list_Revolving loans,NAME_CONTRACT_TYPE_y_list_XNA,WEEKDAY_APPR_PROCESS_START_y_list_FRIDAY,WEEKDAY_APPR_PROCESS_START_y_list_MONDAY,WEEKDAY_APPR_PROCESS_START_y_list_SATURDAY,WEEKDAY_APPR_PROCESS

In [19]:
cat = df.select_dtypes(include='object').columns
lb = LabelEncoder()
for col in cat:
    df[col] = lb.fit_transform(df[col])

In [6]:
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold,RandomizedSearchCV,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,recall_score,f1_score,precision_score,confusion_matrix,r2_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
from xgboost import XGBClassifier
import lightgbm as lgm

In [7]:
def split(dataframe):
    X = dataframe.drop('TARGET',axis=1)
    y = dataframe['TARGET']
    y = y.astype('int')

    X_train1,X_test,y_train1,y_test = train_test_split(X,y,test_size=0.3,random_state=0)

    X_train,X_val,y_train,y_val = train_test_split(X_train1,y_train1,test_size=0.2,random_state=0)
    return X_train1,X_test,y_train1,y_test,X_train,X_val,y_train,y_val

def fit_predict(model):
    model.fit(X_train,y_train)

    y_pred = model.predict(X_val)

    print("Accuracy = {0} \nRecall {1} \nPrecision {2} \nF1 score {3}".format( 
        accuracy_score(y_val,y_pred),recall_score(y_val,y_pred),
        precision_score(y_val,y_pred),f1_score(y_val,y_pred)))

def cross_validate(model,scoring_metric,folds):  
    skf = StratifiedKFold(folds,shuffle=True,random_state=0)
    scores = cross_val_score(log,X_train1,y_train1,cv=skf,scoring=scoring_metric)

    print(scores)
    print(np.mean(scores))

def feature_importance(model):

    feature_imp =  pd.DataFrame(model.feature_importances_,index=list(df.columns.drop('TARGET')),
              columns=['feature_imp']).sort_values('feature_imp',ascending=False)
    imp_columns = list(feature_imp[feature_imp['feature_imp']>0].index)
    imp_df = df[imp_columns]
    imp_df['TARGET'] = df['TARGET']
    return feature_imp,imp_df

In [24]:
X_train1,X_test,y_train1,y_test,X_train,X_val,y_train,y_val = split(df)

In [192]:
log = LogisticRegression(random_state=42)
fit_predict(log)

Accuracy = 0.9189803940959774 
Recall 0.014889091461561836 
Precision 0.5833333333333334 
F1 score 0.02903703703703704


In [193]:
log.fit(X_train,y_train)
y_pred = log.predict(X_val)

In [ ]:
y_pred = pd.Series(y_pred,index=list(y_val.index))
length = int(np.round(len(y_val)/10))
ks = pd.DataFrame(columns=['Total_customers','Good','Bad'])
i = 0
while i<=len(y_val):
  good = y_pred[i:i+length].value_counts()[0]
  bad = y_pred[i:i+length].value_counts()[1]
  ks = ks.append({'Total_customers':int(length),'Good':good,'Bad':bad},ignore_index=True)
  i = i+length

In [ ]:
ks

,Total_customers,Good,Bad
0,4045,4036,9
1,4045,4033,12
2,4045,4044,1
3,4045,4036,9
4,4045,4035,10
5,4045,4033,12
6,4045,4039,6
7,4045,4041,4
8,4045,4036,9
9,4045,4027,15


In [13]:
gb = GradientBoostingClassifier(random_state=0)

fit_predict(gb)

Accuracy = 0.9193018023586422 
Recall 0.017927681555758128 
Precision 0.6483516483516484 
F1 score 0.03489059727971615


In [14]:
gb_feature_imp,imp_df = feature_importance(gb)

In [15]:
imp_columns = list(gb_feature_imp[:20].index)
imp_df = df[imp_columns]
imp_df['TARGET'] = df['TARGET']

In [16]:
X_train1,X_test,y_train1,y_test,X_train,X_val,y_train,y_val = split(imp_df)

In [ ]:
gb.get_params

<bound method BaseEstimator.get_params of GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)>

In [ ]:
gb = GradientBoostingClassifier(random_state=0)
fit_predict(gb)

Accuracy = 0.9190792889460281 
Recall 0.0176238225463385 
Precision 0.5918367346938775 
F1 score 0.03422838595455886


In [ ]:
gb.fit(X_train,y_train)
y_pred = gb.predict(X_val)

In [ ]:
y_pred = pd.Series(y_pred,index=list(y_val.index))
length = int(np.round(len(y_val)/10))
ks = pd.DataFrame(columns=['Total_customers','Good','Bad'])
i = 0
while i<=len(y_val):
  good = y_pred[i:i+length].value_counts()[0]
  bad = y_pred[i:i+length].value_counts()[1]
  ks = ks.append({'Total_customers':int(length),'Good':good,'Bad':bad},ignore_index=True)
  i = i+length

In [ ]:
ks

,Total_customers,Good,Bad
0,4045,4032,13
1,4045,4032,13
2,4045,4040,5
3,4045,4036,9
4,4045,4035,10
5,4045,4036,9
6,4045,4037,8
7,4045,4038,7
8,4045,4038,7
9,4045,4032,10


In [22]:
import re
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [25]:
lgbm = lgm.LGBMClassifier(random_state=0,n_jobs=6)
fit_predict(lgbm)

Accuracy = 0.9187084332583381 
Recall 0.02187784867821331 
Precision 0.5106382978723404 
F1 score 0.04195804195804196


### Hyperparameter Tuning

In [ ]:
gb = GradientBoostingClassifier(random_state=0)

learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, min_impurity_split=None, init=None, random_state=None, max_features=None, verbose=0, max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0

In [ ]:
params = {
    'n_estimators':[150,200],
    'max_depth':[3,4]
}

In [ ]:
random_cv = RandomizedSearchCV(gb,param_distributions=params,n_jobs=6,verbose=1,cv=3,random_state=0)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  20 out of  20 | elapsed: 54.2min finished
[Parallel(n_jobs=6)]: Done  20 out of  20 | elapsed: 54.2min remaining:    0.0s

In [ ]:
random_cv.fit(X_train1,y_train1)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  20 out of  20 | elapsed: 54.2min finished
[Parallel(n_jobs=6)]: Done  20 out of  20 | elapsed: 54.2min remaining:    0.0s


RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                        criterion='friedman_mse',
                                                        init=None,
                                                        learning_rate=0.1,
                                                        loss='deviance',
                                                        max_depth=3,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                 

In [ ]:
random_cv.best_params_

{'n_estimators': 150, 'max_depth': 3}

In [ ]:
gb = GradientBoostingClassifier(random_state=0,n_estimators=150,max_depth=3)

In [ ]:
fit_predict(gb)

Accuracy = 0.9191534600835661 
Recall 0.023093284715891827 
Precision 0.5801526717557252 
F1 score 0.044418468731735834


In [ ]:
application = pd.read_pickle('/content/drive/MyDrive/merged_df.pkl').set_index('SK_ID_CURR')

In [ ]:
application.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,SK_ID_PREV_list,NAME_CONTRACT_TYPE_y_list,AMT_ANNUITY_y_list,AMT_APPLICATION_list,AMT_CREDIT_y_list,AMT_DOWN_PAYMENT_list,AMT_GOODS_PRICE_y_list,WEEKDAY_APPR_PROCESS_START_y_list,HOUR_APPR_PROCESS_START_y_list,FLAG_LAST_APPL_PER_CONTRACT_list,NFLAG_LAST_APPL_IN_DAY_list,RATE_DOWN_PAYMENT_list,RATE_INTEREST_PRIMARY_list,RATE_INTEREST_PRIVILEGED_list,NAME_CASH_LOAN_PURPOSE_list,NAME_CONTRACT_STATUS_list,DAYS_DECISION_list,NAME_PAYMENT_TYPE_list,CODE_REJECT_REASON_list,NAME_TYPE_SUITE_y_list,NAME_CLIENT_TYPE_list,NAME_GOODS_CATEGORY_list,NAME_PORTFOLIO_list,NAME_PRODUCT_TYPE_list,CHANNEL_TYPE_list,SELLERPLACE_AREA_list,NAME_SELLER_INDUSTRY_list,CNT_PAYMENT_list,NAME_YIELD_GROUP_list,PRODUCT_COMBINATION_list,DAYS_FIRST_DRAWING_list,DAYS_FIRST_DUE_list,DAYS_LAST_DUE_1ST_VERSION_list,DAYS_LAST_DUE_list,DAYS_TERMINATION_list,NFLAG_INSURED_ON_APPROVAL_list
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2,2.0,2,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,1.0,[1038818],[Consumer loans],[9251.775],[179055.0],[179055.0],[0.0],[179055.0],[SATURDAY],[9],[Y],[1],[0.0],[nan],[nan],[XAP],[Approved],[-606],[XNA],[XAP],[nan],[New],[Vehicles],[POS],[XNA],[Stone],[500],[Auto technology],[24.0]

In [ ]:
null_values = (application.isna().sum()/len(application)*100).sort_values(ascending=False).head(100)
to_add_cols = list(null_values[(null_values>47)&(null_values<60)].index)

In [ ]:
application = application[to_add_cols]
application.drop('EXT_SOURCE_1',axis=1,inplace=True)

In [ ]:
num = application.select_dtypes(exclude='object').columns
cat = application.select_dtypes(include='object').columns
print(num,"\n",cat)

Index(['LANDAREA_MODE', 'LANDAREA_MEDI', 'LANDAREA_AVG', 'BASEMENTAREA_MODE',
       'BASEMENTAREA_AVG', 'BASEMENTAREA_MEDI', 'NONLIVINGAREA_AVG',
       'NONLIVINGAREA_MEDI', 'NONLIVINGAREA_MODE', 'ELEVATORS_MEDI',
       'ELEVATORS_AVG', 'ELEVATORS_MODE', 'APARTMENTS_MODE', 'APARTMENTS_AVG',
       'APARTMENTS_MEDI', 'ENTRANCES_MODE', 'ENTRANCES_AVG', 'ENTRANCES_MEDI',
       'LIVINGAREA_AVG', 'LIVINGAREA_MEDI', 'LIVINGAREA_MODE',
       'FLOORSMAX_MEDI', 'FLOORSMAX_MODE', 'FLOORSMAX_AVG',
       'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BEGINEXPLUATATION_MODE',
       'YEARS_BEGINEXPLUATATION_AVG', 'TOTALAREA_MODE'],
      dtype='object') 
 Index(['WALLSMATERIAL_MODE', 'HOUSETYPE_MODE', 'EMERGENCYSTATE_MODE'], dtype='object')


In [ ]:
imputer = SimpleImputer(strategy='mean')
application[num] = imputer.fit_transform(application[num])

In [ ]:
imputer = SimpleImputer(strategy='most_frequent')
application[cat] = imputer.fit_transform(application[cat])

In [ ]:
scaler = StandardScaler()
application[num] = scaler.fit_transform(application[num])

In [ ]:
cat

Index(['WALLSMATERIAL_MODE', 'HOUSETYPE_MODE', 'EMERGENCYSTATE_MODE'], dtype='object')

In [ ]:
application = pd.get_dummies(application)

In [ ]:
application = application[application.index.isin(df.index)]

In [ ]:
application.shape,df.shape

((288907, 40), (288907, 229))

In [ ]:
df = df.merge(application,right_index=True,left_index=True)